In [1]:
import pandas as pd
import nltk
import re
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
import warnings
warnings.filterwarnings("ignore")
import nltk
import numpy as np

nltk.download('stopwords')
stopwords_set =  set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/caokeyi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('go_emotions_dataset.csv')

total_rows = df.shape[0]
print("Total Rows: ", total_rows)

Total Rows:  211225


In [3]:
positive_col = ['admiration','amusement','approval','caring','excitement','gratitude','joy','love','optimism','relief','pride','desire']#12
negative_col = ['sadness','disgust','fear','embarrassment','disapproval','disappointment','grief','annoyance','anger','nervousness','remorse']#11
neutral_col = ['curiosity','realization','surprise','confusion','neutrality']#4+1=5

df.rename(columns={'neutral': 'neutrality'}, inplace=True)

In [4]:
df['positive'] = df[positive_col].apply(lambda row: ''.join(map(str, row)), axis=1)
df['positive'] = df['positive'].apply(lambda x: 1 if '1' in x else 0)
df['neutral'] = df[neutral_col].apply(lambda row: ''.join(map(str, row)), axis=1)
df['neutral'] = df['neutral'].apply(lambda x: 1 if '1' in x else 0)

df.head()

id                                               text  \
0  eew5j0j                                    That game hurt.   
1  eemcysk   >sexuality shouldn’t be a grouping category I...   
2  ed2mah1     You do right, if you don't care then fuck 'em!   
3  eeibobj                                 Man I love reddit.   
4  eda6yn6  [NAME] was nowhere near them, he was by the Fa...   

   example_very_unclear  admiration  amusement  anger  annoyance  approval  \
0                 False           0          0      0          0         0   
1                  True           0          0      0          0         0   
2                 False           0          0      0          0         0   
3                 False           0          0      0          0         0   
4                 False           0          0      0          0         0   

   caring  confusion  ...  optimism  pride  realization  relief  remorse  \
0       0          0  ...         0      0            0       0        0   
1       0          0  ...         0      0            0       0        0   
2       0          0  ...         0      0            0       0        0   
3       0          0  ...         0      0            0       0        0   
4       0          0  ...         0      0            0       0        0   

   sadness  surprise  neutrality  positive  neutral  
0        1         0           0         0        0  
1        0         0           0         0        0  
2        0         0           1         0        1  
3        0         0           0         1        0  
4        0         0           1         0        1  

[5 rows x 33 columns]

In [5]:
columns_to_drop = positive_col + neutral_col
df.drop(columns=columns_to_drop, inplace=True)

df.head()

id                                               text  \
0  eew5j0j                                    That game hurt.   
1  eemcysk   >sexuality shouldn’t be a grouping category I...   
2  ed2mah1     You do right, if you don't care then fuck 'em!   
3  eeibobj                                 Man I love reddit.   
4  eda6yn6  [NAME] was nowhere near them, he was by the Fa...   

   example_very_unclear  anger  annoyance  disappointment  disapproval  \
0                 False      0          0               0            0   
1                  True      0          0               0            0   
2                 False      0          0               0            0   
3                 False      0          0               0            0   
4                 False      0          0               0            0   

   disgust  embarrassment  fear  grief  nervousness  remorse  sadness  \
0        0              0     0      0            0        0        1   
1        0              0     0      0            0        0        0   
2        0              0     0      0            0        0        0   
3        0              0     0      0            0        0        0   
4        0              0     0      0            0        0        0   

   positive  neutral  
0         0        0  
1         0        0  
2         0        1  
3         1        0  
4         0        1

In [6]:
count_positive = df[df['positive'] == 1].shape[0]
count_neutral = df[df['neutral'] == 1].shape[0]
count_total = ((df['positive'] == 1) | (df['neutral'] == 1)).sum()

print("positive: ", count_positive)
print("neutral:", count_neutral)
print("total:", count_total)

positive:  82938
neutral: 84580
total: 161823


In [7]:
def keep_row(row):
    if row['positive'] == 1 or row['neutral'] == 1:
        return np.random.choice([True, False], p=[1/17, 16/17])
    else:
        return True
    
df = df[df.apply(keep_row, axis=1)]


In [8]:
count_positive = df[df['positive'] == 1].shape[0]
count_neutral = df[df['neutral'] == 1].shape[0]
count_total = ((df['positive'] == 1) | (df['neutral'] == 1)).sum()

print("positive: ", count_positive)
print("neutral:", count_neutral)
print("total:", count_total)

new_total_rows = df.shape[0]
print("New Total Rows: ", new_total_rows)
print("Dropped Rows: ", total_rows - new_total_rows)

positive:  4835
neutral: 4911
total: 9429
New Total Rows:  58831
Dropped Rows:  152394


In [9]:
def clean(text):
    text = str(text).lower()# 转换成小写
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[^a-zA-Z\s]', '', text)# filter non English char
    text = [word for word in text.split(' ') if word not in stopwords_set]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

df['token'] = df['text'].apply(clean)
df.head()

id                                               text  \
0   eew5j0j                                    That game hurt.   
1   eemcysk   >sexuality shouldn’t be a grouping category I...   
6   eczuekb  He isn't as big, but he's still quite popular....   
11  ee52cjs  I wanted to downvote this, but it's not your f...   
13  ee9vw9t                                       That is odd.   

    example_very_unclear  anger  annoyance  disappointment  disapproval  \
0                  False      0          0               0            0   
1                   True      0          0               0            0   
6                  False      0          0               0            1   
11                 False      0          0               1            0   
13                 False      0          0               1            0   

    disgust  embarrassment  fear  grief  nervousness  remorse  sadness  \
0         0              0     0      0            0        0        1   
1         0              0     0      0            0        0        0   
6         0              0     0      0            0        0        0   
11        0              0     0      0            0        0        0   
13        1              0     0      0            0        0        0   

    positive  neutral                                              token  
0          0        0                                          game hurt  
1          0        0   sexual shouldnt group categori make differ ot...  
6          0        0  isnt big hes still quit popular ive heard thin...  
11         0        0                            want downvot fault homi  
13         0        0                                                odd

In [10]:
df.to_csv('emotions_token_removePositive.csv',index = False)